In [1]:
import os, sys, time
from time import sleep
from pathlib import Path
import fnmatch, glob, shutil
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import tempfile, logging
from zipfile import ZipFile

In [2]:
from PyPDF2 import PdfFileWriter, PdfFileReader

In [3]:
# SETUP LOGGER
try: #{
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s : \n\t\t\t\t [MESG] : %(message)s \n\t\t\t\t [LINE] : %(lineno)s',
                        datefmt='%Y-%m-%d-%H%M%S',
                        filemode='a')
#}
except: #{
    errorMessage = str(sys.exc_info()[0]) + "\n\t\t"
    errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
    errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
    exc_type, exc_obj, exc_tb = sys.exc_info()
    fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
    typeE = str("TYPE : " + str(exc_type))
    fileE = str("FILE : " + str(fname))
    lineE = str("LINE : " + str(exc_tb.tb_lineno))
    messageE = str("MESG : " + "\n" + str(errorMessage) + "\n")
    print("\n" + typeE +
          "\n" + fileE +
          "\n" + lineE +
          "\n" + messageE)
#}
else: #{
    print("FIN...")
#}

FIN...


In [4]:
# CREATE TIME STAMP (string) FOR ZIP FOLDER
ts = pd.Timestamp.now()
ts_str = str(ts)
ts_str = ts_str[:10]
print(ts_str)

2019-07-24


In [5]:
# DIRECTORIES TO SCAN
in_directory_1 = "F:/APPS/CofA/"
in_directory_2 = "G:/C of A's/#Email Node/"

In [6]:
out_directory = "."

In [7]:
zip_dir_path = os.path.join(out_directory, ts_str)
print(zip_dir_path)

.\2019-07-24


In [8]:
# if 'zip_folder' DOES NOT YET EXIST!
if not os.path.exists(zip_dir_path): #{
    # MAKE IT EXIST!
    os.makedirs(zip_dir_path)
#}

In [9]:
# RESET/RE-ESTABLISH 'out_directory'
out_directory = Path(zip_dir_path)

In [10]:
watermark = "C:/data/inbound/Agilent_CofA_Letterhead_03-21-19.pdf"

## *DEFINE FUNCTIONS*

In [11]:
def create_watermark(input_pdf, output, watermark): #{
    try: #{
        watermark_obj = PdfFileReader(watermark)
        watermark_page = watermark_obj.getPage(0)
        
        pdf_reader = PdfFileReader(input_pdf)
        pdf_writer = PdfFileWriter()
        
        # Watermark all the pages
        for page in range(pdf_reader.getNumPages()): #{
            page = pdf_reader.getPage(page)
            page.mergePage(watermark_page)
            pdf_writer.addPage(page)
        #}
        
        with open(output, 'wb') as out: #{
            pdf_writer.write(out)
        #}
    #}
    except: #{
        errorMessage = str(sys.exc_info()[0]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        typeE = str("TYPE : " + str(exc_type))
        fileE = str("FILE : " + str(fname))
        lineE = str("LINE : " + str(exc_tb.tb_lineno))
        messageE = str("MESG : " + "\n" + str(errorMessage) + "\n")
        logging.info("\n" + typeE +
              "\n" + fileE +
              "\n" + lineE +
              "\n" + messageE)
    #}
    else: #{
        logging.info("[Watermark-PDF] FIN...")
    #}
    return
#}

In [12]:
def get_all_file_paths(directory): #{
    
    # initializing empty file paths list
    file_paths = []
    
    # crawling through directory and subdirectories
    for root, directories, files in os.walk(directory): #{
        for filename in files: #{
            # join the two strings in order to form the full filepath
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)
        #}
    #}
    
    # returning all file paths
    return file_paths

#}

In [13]:
in_file = "data/batch_list.csv"

In [14]:
df_batch= pd.read_csv(in_file, header=None, names=['Lot_No'], engine='python')

In [15]:
df_batch.head(5)

,Lot_No
0,47995N-1
1,DWM-544-1
2,DWM-580-1
3,DWM-583-1
4,DWM-588-1


In [16]:
test_glob = str(in_directory_1 + "*")
print(test_glob)

F:/APPS/CofA/*


In [17]:
# counter
x = 0

for lot_no in df_batch.itertuples(): #{
    logging.info("============== \n " + str(lot_no[1]) + "\n==============")
    # CHECK AND CREATE NEW DIR PATH
    new_dir_path = os.path.join(out_directory, str(lot_no[1]))
    # IF 'lot_no' DIRECTORY DOES NOT EXIST:
    if not os.path.exists(new_dir_path): #{
        # MAKE IT EXIST!
        os.makedirs(new_dir_path)
    #}
    # CREATE TEMP DIR/FOLDER TO WORK INSIDE OF:
    with tempfile.TemporaryDirectory() as directory_name: #{
        # was: most_recent = ""
        the_dir = Path(directory_name)
        logging.info("TEMP_DIRECTORY == " + str(the_dir))
        # GLOB AND ITERATURE THROUGH EACH FILE MATCHING NAME ETC
        for name in sorted(glob.glob(in_directory_1 + "*" + str(lot_no[1]) + "*")): #{
            #print(name)
            # CREAT PATH VARIABLE
            pdf_path = Path(str(name))
            logging.info(os.path.exists(pdf_path))
            # GET BASE NAME (FILE NAME)
            file_name = os.path.basename(pdf_path)
            # CREATE TEMP PATH
            temp_path = os.path.join(the_dir, file_name)
            # COPY FILE TO NEW LOCATION
            shutil.copy2(pdf_path, temp_path)
            # CREATE WATERMARK ON NEW FILE
            # (OUTPUT == out_directory + str(lot_no[1]) + filename)
            create_watermark(input_pdf=temp_path,
                            output=os.path.join(new_dir_path, file_name),
                            watermark=watermark)
            # increase count
            x += 1
        #}
    #}
    logging.info("count == " + (str(x)))
    logging.info("Directory exists after?", str(the_dir.exists()))
    logging.info("Contents after:", str(list(the_dir.glob("*"))))
    logging.info("told you so")
#}

2019-07-24-162341 : 
				 [MESG] : ============== 
 47995N-1
				 [LINE] : 5
2019-07-24-162341 : 
				 [MESG] : TEMP_DIRECTORY == C:\Users\derbates\AppData\Local\Temp\tmp6x6u1aif 
				 [LINE] : 17
2019-07-24-162342 : 
				 [MESG] : count == 0 
				 [LINE] : 39
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 1034, in emit
    msg = self.format(record)
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 880, in format
    return fmt.format(record)
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 619, in format
    record.message = record.getMessage()
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 380, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "C:\Users\derbates\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\derbates\anaconda3\li

2019-07-24-162342 : 
				 [MESG] : ============== 
 DWM-544-1
				 [LINE] : 5
2019-07-24-162342 : 
				 [MESG] : TEMP_DIRECTORY == C:\Users\derbates\AppData\Local\Temp\tmpgptyyjmt 
				 [LINE] : 17
2019-07-24-162344 : 
				 [MESG] : True 
				 [LINE] : 23
2019-07-24-162344 : 
				 [MESG] : [Watermark-PDF] FIN... 
				 [LINE] : 36
2019-07-24-162344 : 
				 [MESG] : True 
				 [LINE] : 23
2019-07-24-162344 : 
				 [MESG] : [Watermark-PDF] FIN... 
				 [LINE] : 36
2019-07-24-162344 : 
				 [MESG] : count == 2 
				 [LINE] : 39
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 1034, in emit
    msg = self.format(record)
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 880, in format
    return fmt.format(record)
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 619, in format
    record.message = record.getMessage()
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 

2019-07-24-162344 : 
				 [MESG] : ============== 
 DWM-580-1
				 [LINE] : 5
2019-07-24-162344 : 
				 [MESG] : TEMP_DIRECTORY == C:\Users\derbates\AppData\Local\Temp\tmprcetz_3e 
				 [LINE] : 17
2019-07-24-162345 : 
				 [MESG] : True 
				 [LINE] : 23
2019-07-24-162346 : 
				 [MESG] : [Watermark-PDF] FIN... 
				 [LINE] : 36
2019-07-24-162346 : 
				 [MESG] : True 
				 [LINE] : 23
2019-07-24-162347 : 
				 [MESG] : [Watermark-PDF] FIN... 
				 [LINE] : 36
2019-07-24-162347 : 
				 [MESG] : count == 4 
				 [LINE] : 39
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 1034, in emit
    msg = self.format(record)
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 880, in format
    return fmt.format(record)
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 619, in format
    record.message = record.getMessage()
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 

2019-07-24-162347 : 
				 [MESG] : ============== 
 DWM-583-1
				 [LINE] : 5
2019-07-24-162347 : 
				 [MESG] : TEMP_DIRECTORY == C:\Users\derbates\AppData\Local\Temp\tmpwgsceqnr 
				 [LINE] : 17
2019-07-24-162348 : 
				 [MESG] : True 
				 [LINE] : 23
2019-07-24-162349 : 
				 [MESG] : [Watermark-PDF] FIN... 
				 [LINE] : 36
2019-07-24-162349 : 
				 [MESG] : count == 5 
				 [LINE] : 39
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 1034, in emit
    msg = self.format(record)
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 880, in format
    return fmt.format(record)
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 619, in format
    record.message = record.getMessage()
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 380, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "C:\Us

2019-07-24-162349 : 
				 [MESG] : ============== 
 DWM-588-1
				 [LINE] : 5
2019-07-24-162349 : 
				 [MESG] : TEMP_DIRECTORY == C:\Users\derbates\AppData\Local\Temp\tmp_qep17jq 
				 [LINE] : 17
2019-07-24-162350 : 
				 [MESG] : count == 5 
				 [LINE] : 39
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 1034, in emit
    msg = self.format(record)
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 880, in format
    return fmt.format(record)
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 619, in format
    record.message = record.getMessage()
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 380, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "C:\Users\derbates\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\derbates\anaconda3\l

...

2019-07-24-162851 : 
				 [MESG] : ============== 
 WRK-151O
				 [LINE] : 5
2019-07-24-162851 : 
				 [MESG] : TEMP_DIRECTORY == C:\Users\derbates\AppData\Local\Temp\tmpcrfmtryr 
				 [LINE] : 17
2019-07-24-162852 : 
				 [MESG] : True 
				 [LINE] : 23
2019-07-24-162853 : 
				 [MESG] : [Watermark-PDF] FIN... 
				 [LINE] : 36
2019-07-24-162853 : 
				 [MESG] : True 
				 [LINE] : 23
2019-07-24-162853 : 
				 [MESG] : [Watermark-PDF] FIN... 
				 [LINE] : 36
2019-07-24-162853 : 
				 [MESG] : count == 498 
				 [LINE] : 39
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 1034, in emit
    msg = self.format(record)
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 880, in format
    return fmt.format(record)
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 619, in format
    record.message = record.getMessage()
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line

2019-07-24-162853 : 
				 [MESG] : ============== 
 WRK-151P
				 [LINE] : 5
2019-07-24-162853 : 
				 [MESG] : TEMP_DIRECTORY == C:\Users\derbates\AppData\Local\Temp\tmprxxs55ei 
				 [LINE] : 17
2019-07-24-162855 : 
				 [MESG] : count == 498 
				 [LINE] : 39
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 1034, in emit
    msg = self.format(record)
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 880, in format
    return fmt.format(record)
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 619, in format
    record.message = record.getMessage()
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 380, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "C:\Users\derbates\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\derbates\anaconda3\

2019-07-24-162855 : 
				 [MESG] : ============== 
 WRK-151Q
				 [LINE] : 5
2019-07-24-162855 : 
				 [MESG] : TEMP_DIRECTORY == C:\Users\derbates\AppData\Local\Temp\tmpic38nvy1 
				 [LINE] : 17
2019-07-24-162856 : 
				 [MESG] : count == 498 
				 [LINE] : 39
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 1034, in emit
    msg = self.format(record)
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 880, in format
    return fmt.format(record)
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 619, in format
    record.message = record.getMessage()
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 380, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "C:\Users\derbates\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\derbates\anaconda3\

2019-07-24-162856 : 
				 [MESG] : ============== 
 WRK-151R
				 [LINE] : 5
2019-07-24-162856 : 
				 [MESG] : TEMP_DIRECTORY == C:\Users\derbates\AppData\Local\Temp\tmpz_vkwj7q 
				 [LINE] : 17
2019-07-24-162858 : 
				 [MESG] : count == 498 
				 [LINE] : 39
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 1034, in emit
    msg = self.format(record)
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 880, in format
    return fmt.format(record)
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 619, in format
    record.message = record.getMessage()
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 380, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "C:\Users\derbates\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\derbates\anaconda3\

2019-07-24-162858 : 
				 [MESG] : ============== 
 WRK-151S
				 [LINE] : 5
2019-07-24-162858 : 
				 [MESG] : TEMP_DIRECTORY == C:\Users\derbates\AppData\Local\Temp\tmpvlwm0i2v 
				 [LINE] : 17
2019-07-24-162859 : 
				 [MESG] : count == 498 
				 [LINE] : 39
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 1034, in emit
    msg = self.format(record)
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 880, in format
    return fmt.format(record)
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 619, in format
    record.message = record.getMessage()
  File "C:\Users\derbates\anaconda3\lib\logging\__init__.py", line 380, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "C:\Users\derbates\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\derbates\anaconda3\

In [18]:
# TRY AND ZIP FOLDER
try: #{
    # path to folder which needs to be zipped
    directory = zip_dir_path
    
    # calling function to get all file paths in the directory
    file_paths = get_all_file_paths(directory)
    
    # printing list of all files to be zipped
    print('Following files will be zipped:')
    for file_name in file_paths: #{
        print(file_name)
    #}
    
    zip_name = str(ts_str + ".zip")
    
    # writing files to a zipfile
    with ZipFile(zip_name, 'w') as zip: #{
        # writing each file one by one
        for file in file_paths: #{
            zip.write(file)
        #}
    #}
    
    print("All Files zipped successfully!")
#}
except: #{
    errorMessage = str(sys.exc_info()[0]) + "\n\t\t"
    errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
    errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
    exc_type, exc_obj, exc_tb = sys.exc_info()
    fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
    typeE = str("TYPE : " + str(exc_type))
    fileE = str("FILE : " + str(fname))
    lineE = str("LINE : " + str(exc_tb.tb_lineno))
    messageE = str("MESG : " + "\n" + str(errorMessage) + "\n")
    logging.info("\n" + typeE +
                 "\n" + fileE +
                 "\n" + lineE +
                 "\n" + messageE)
#}
else: #{
    logging.info("[zip-folder] FIN...")
#}

Following files will be zipped:
.\2019-07-24\DWM-544-1\DWM-544-1@0006460286.pdf
.\2019-07-24\DWM-544-1\DWM-544-1@0006476293.pdf
.\2019-07-24\DWM-580-1\DWM-580-1@0006472370.pdf
.\2019-07-24\DWM-580-1\DWM-580-1@0006472371.pdf
.\2019-07-24\DWM-583-1\DWM-583-1@CG-3973.PDF
.\2019-07-24\IBF-1245\IBF-1245@CL-1912.pdf
.\2019-07-24\IBF-1245\IBF-1245@CL-3705.pdf
.\2019-07-24\IBF-1245\IBF-1245@CM-2198.pdf
.\2019-07-24\IBF-1245\IBF-1245@CM-4282.pdf
.\2019-07-24\IBF-1245\IBF-1245@CM-5863.pdf
.\2019-07-24\IBF-1245\IBF-1245@CP-3834.pdf
.\2019-07-24\IBF-1245\IBF-1245@CP-4256.pdf
.\2019-07-24\IBF-1245\IBF-1245@CP-5853.pdf
.\2019-07-24\IBF-1245\IBF-1245@CR-0980.pdf
.\2019-07-24\IBF-1245\IBF-1245@CR-2498.pdf
.\2019-07-24\IBF-1245\IBF-1245@CR-3502.pdf
.\2019-07-24\IBF-1245\IBF-1245@CR-5266.pdf
.\2019-07-24\IBF-1245\IBF-1245@CR-6109.pdf
.\2019-07-24\IBF-1245\IBF-1245@CS-1746.pdf
.\2019-07-24\IBF-1245\IBF-1245@CS-5595.pdf
.\2019-07-24\IBF-1245\IBF-1245@CT-0400.pdf
.\2019-07-24\IBF-1245\IBF-1245@CT-1199.pdf


2019-07-24-162901 : 
				 [MESG] : [zip-folder] FIN... 
				 [LINE] : 43


All Files zipped successfully!
